# UGRID Tools: MO/UCAR 06 April 2022

<h1 align="center"> <a href="https://github.com/bjlittle/geovista"> <img src="resources/geovistalogo.png" alt="GeoVista" width="200"> </a></h1>

<h3 align="center"> Cartographic rendering and mesh analytics powered by <a href="https://docs.pyvista.org/index.html">PyVista</a></h3>

In this notebook we'll introduce `geovista`. A new, exciting open-source Scientific Python package being developed by the AVD Team, which leverages the benefits of the Python package `pyvista` to offer geospatial rendering and unstructured mesh analytics.

`geovista` delivers a paradigm shift in performance, thanks to building upon [VTK](https://vtk.org/), which is implemented in **C++** and supports **GPU hardware accelerated rendering** through [OpenGL](https://www.opengl.org//).

## What's Coming Up...

We'll give a whirlwind tour of the following `geovista` topics the underpin our planar projection support:

1. **Exploring our Data with GeoVista**
1. **Using a KDTree with an LFRic Cubed-Sphere Mesh**

## Notebook Configuration

First, let's configure the notebook with the packages that we need...

In [ ]:
import math
from pathlib import Path
from typing import Optional

from cartopy.crs import PlateCarree
import geovista as gv
import geovista.theme
import iris
from iris.experimental.ugrid import PARSE_UGRID_ON_LOAD
import netCDF4
import pyvista as pv


print(f"{iris.__version__=}")
print(f"{gv.__version__=}")

#### Jupyter Notebook 3D Modules

Module       | Jupyterlab 3 | Rendering Location | Backend | Requires Framebuffer | Demo
:-----------:| :----------: | :----------------: | :-----: | :------------------: | :--:
`ipygany`    | Yes          | Client             | threejs | No                   | 
`ipyvtklink` | Yes          | Server             | vtk     | Yes                  | ✔️
`itkwidgets` | No           | Client             | vtk.js  | Yes                  |
`panel`      | Yes          | Client             | vtk.js  | Yes                  |
`pythreejs`  | Yes          | Client             | threejs | No                   | ✔️


See [PyVista Jupyter Notebook Plotting](https://docs.pyvista.org/user-guide/jupyter/index.html) for further details.

In [ ]:
pv.set_jupyter_backend("pythreejs")

In [ ]:
BASE_DIR = Path("./data")

## Utility Functions

For convenience sake, create some utility functions to load assorted data that is unstructured...

In [ ]:
def load_ugrid(
    fname: str,
    data: Optional[bool] = False,
    constraint: Optional[str] = None,
    verbose: Optional[bool] = False
) -> pv.PolyData:
    fname = BASE_DIR / fname
    with PARSE_UGRID_ON_LOAD.context():
        cube = iris.load_cube(fname, constraint=constraint)
        
    if cube.ndim > 1:
        cube = cube[(0,) * (cube.ndim - 1)]
    
    if verbose:
        print(cube)
    
    data = cube.data if data else None
        
    face_node = cube.mesh.face_node_connectivity
    indices = face_node.indices_by_location()
    lons, lats = cube.mesh.node_coords

    mesh = gv.Transform.from_unstructured(
        lons.points,
        lats.points,
        indices,
        data=data,
        start_index=face_node.start_index,
        name=cube.name(),
    )

    if data is None:
        mesh.active_scalars_name = None
    
    return mesh

In [ ]:
def load_lam(
    data: Optional[bool] = True,
    verbose: Optional[bool] = False
) -> pv.PolyData:
    return load_ugrid(
        "lam.nc",
        data=data,
        constraint="air_potential_temperature",
        verbose=verbose,
    )

In [ ]:
def load_sst(
    data: Optional[bool] = True,
    verbose: Optional[bool] = False
) -> pv.PolyData:
    return load_ugrid(
        "sst.nc",
        data=data,
        verbose=verbose,
    )

In [ ]:
def load_icosahedral(
    data: Optional[bool] = True, 
    verbose: Optional[bool] = False
) -> pv.PolyData:
    # see https://github.com/SciTools/cartopy/issues/2016
    ds = netCDF4.Dataset(BASE_DIR / "icosahedral.nc")
    lons = ds.variables["bounds_lon_i"][:]
    lats = ds.variables["bounds_lat_i"][:]
    data = ds.variables["phis"][:] if data else None
    
    if verbose:
        print(ds)
    
    mesh = gv.Transform.from_unstructured(
        lons,
        lats,
        lons.shape,
        data=data,
    )
    
    if data is None:
        mesh.active_scalars_name = None
        
    return mesh

In [ ]:
def load_moisture(
    data: Optional[bool] = True,
    verbose: Optional[bool] = False
) -> pv.PolyData:
    return load_ugrid(
        "soil.nc",
        data=data,
        constraint="soil_moisture",
        verbose=verbose,
    )

In [ ]:
def info(mesh: pv.PolyData) -> None:
    print(f"The mesh is a C{int(math.sqrt(mesh.n_cells / 6))}, with 6 panels, {int(mesh.n_cells / 6):,d} cells per panel, and {mesh.n_cells:,d} cells.")

## Exploring our Data with GeoVista

### LFRic: Local Area Model (LAM) Air Potential Temperature

In [ ]:
lam = load_lam(verbose=True)

In [ ]:
lam

Our `mesh` is **Coordinate Reference System** (CRS) aware, thanks to the awesomeness of `pyproj` 🥳...

In [ ]:
lam["gvCRS"]

In [ ]:
crs = gv.from_wkt(lam)
crs

In [ ]:
crs.datum

In [ ]:
crs.ellipsoid

In [ ]:
crs.prime_meridian

In [ ]:
crs.is_geographic

Now let's render the **LAM** mesh using `geovista`...

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_mesh(lam, cmap="balance", show_edges=False)
plotter.add_coastlines(resolution="110m", color="white")
plotter.add_base_layer(texture=gv.natural_earth_hypsometric())

plotter.show()

Now let's explore some **planar projections** of our **LAM** data...

In [ ]:
plotter = gv.GeoPlotter(crs="+proj=moll")

plotter.add_base_layer(texture=gv.natural_earth_hypsometric())
plotter.add_mesh(lam, cmap="balance", show_edges=False)
plotter.view_xy()

plotter.show()

### LFRic: Cubed-Sphere C48 Sea Surface Temperature

Let's explore some cubed-sphere **sea surface temperature** data with a **land mask**...

In [ ]:
sst = load_sst(verbose=True)

In [ ]:
sst

In [ ]:
info(sst)

In [ ]:
sst_holy = sst.threshold()

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_base_layer(texture=gv.natural_earth_hypsometric(), zlevel=-5)
plotter.add_coastlines(resolution="10m", color="white")
plotter.add_mesh(sst_holy, cmap="balance", show_edges=True)

plotter.show()

### Unstructured Hexagonal Mesh

We have support to render generic unstructured meshes, such as this **hexagonal** mesh...

In [ ]:
mesh = load_icosahedral(verbose=True)

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_mesh(mesh, show_edges=True)

plotter.show()

## LFRic: Cubed-Sphere C192 Soil Moisture

Create a `pyvista.PolyData` mesh using `geovista`...

In [ ]:
moisture = load_moisture(verbose=True)

In [ ]:
!ncdump -h ./data/soil.nc

In [ ]:
moisture

In [ ]:
info(moisture)

Now using `geovista`, let's plot the cubed-sphere mesh...

In [ ]:
cmap = "bwy"

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_mesh(moisture, cmap=cmap, show_edges=True)
plotter.add_coastlines(resolution="10m", color="white")

plotter.show()

In [ ]:
def to_centers(mesh: pv.PolyData) -> pv.PolyData:
    tmp = mesh.copy()
    tmp.clear_cell_data()
    tmp.clear_point_data()
    tmp.clear_field_data()
    return tmp.cell_centers()

In [ ]:
moisture_centers = to_centers(moisture)

In [ ]:
%timeit to_centers(moisture)

In [ ]:
moisture_centers

In [ ]:
from pykdtree.kdtree import KDTree

from geovista.common import to_xyz


def find_nearest(tree, points, poi, k):
    # lat/lon to xyz
    xyz = to_xyz(*poi)
    print(f"{poi=}")
    print(f"{xyz=}")
    
    # find the k nearest euclidean neighbours
    dist, idxs = tree.query(xyz, k=k)
    
    print(f"{dist=}")
    print(f"{idxs=}")
    
    if idxs.ndim > 1:
        idxs = idxs[0]
    
    # retieve the associated xyz points of the k nearest neighbours
    nearest = points[idxs]
    
    return xyz, nearest, idxs

In [ ]:
# define some points-of-interest (poi) in (lon, lat) order
north_pole = 0, 90
london = -0.1276, 51.5072
new_york = -74.006, 40.7128

In [ ]:
points = moisture_centers.points
tree = KDTree(points)

poi = london

xyz, nearest, idxs = find_nearest(tree, points, poi, k=10)

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_mesh(moisture, cmap=cmap, show_edges=True)

plotter.add_points(xyz, render_points_as_spheres=True, color="yellow", point_size=.5)
plotter.add_points(moisture_centers, color="black", point_size=1)
plotter.add_points(nearest, render_points_as_spheres=True, color="red", point_size=.5)

plotter.add_coastlines(resolution="10m")

plotter.show(jupyter_backend="pythreejs")

Same again, but this time using the `ipyvtklink` backend renderer...

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_mesh(moisture, cmap=cmap, show_edges=True)

plotter.add_points(xyz, render_points_as_spheres=True, color="yellow", point_size=5)
plotter.add_points(moisture_centers, color="black", point_size=1)
plotter.add_points(nearest, render_points_as_spheres=True, color="red", point_size=5)

plotter.add_coastlines(resolution="10m")

plotter.show(jupyter_backend="ipyvtklink")